In [1]:
# auto-reaload for quick debugging
%load_ext autoreload
%autoreload 2

import ArdLink as al

In [2]:
"""Config"""

port = 'COM2' # port to connect to device

In [3]:
# Bring digital pin 13 (LED) high

# This example executes instructions immediately as Python executes

device = al.Device(port, 9600, timeout=0.05)

pin = 13

device.execute(al.PinMode(pin, 0x0)) # set pinmode at pin 13 to output
device.execute(al.Set(al.device.DATA_BUFFER_OFFSET + 0, 0x1)) # write 1 to addr 0 of data buffer
device.execute(al.DigitalWrite(pin, al.device.DATA_BUFFER_OFFSET + 0)) # write addr 0 of data buffer to digital pin 13


device.close()

In [9]:
# Pulse pin 13 (LED) on a 1 sec sec cycle

# This example is loaded into the instruction buffer then executed
device = al.Device(port, 9600, timeout=0.05)

pin = 13 # digital LED pin
loop_label = 0x00
period = 1000 # ms

device.write_routine([
    al.PinMode(pin, 0x0),
    al.Set(al.device.DATA_BUFFER_OFFSET + 0, 0x1),          # set addr 0 to high const
    al.Set(al.device.DATA_BUFFER_OFFSET + 1, 0x0),          # set addr 1 to low const
    al.Set(al.device.DATA_BUFFER_OFFSET + 2, period//2),    # set addr 2 to half cycle period
    al.Label(loop_label),                                   # set loop label
    al.ReadInstr(),                                         # Check for instructions. This allows us to send a halt command
    al.DigitalWrite(pin, al.device.DATA_BUFFER_OFFSET + 0), # set digital pin to val at addr 0
    al.Delay(al.device.DATA_BUFFER_OFFSET + 2),             # delay for half cycle period
    al.DigitalWrite(pin, al.device.DATA_BUFFER_OFFSET + 1),
    al.Delay(al.device.DATA_BUFFER_OFFSET + 2),
    al.Goto(loop_label)                                     # goto beginning of loop
    ])

device.run_routine()

device.close()